# Modeling + Design

Notes:

- Rand scripts back-to-back to ensure compatability. 
- This notebook automatically tries to grab the most recent artifact from `/tmp/model_artifacts_*`
    - You may need to update this or hand copy the artifact between `train.py` and `generate.py`
- Settings (mostly for training) are adjusted for fast testing and will not reproduce similar results to the paper. 
- You will need to modify hyperparameters yourself. An obvious change is to set `--min_epochs 60` and `--max_epochs 200`.

## Deploy training
Deposit model in `/tmp/`. The progress bar doesn't play nice with the notebook, so you'll have to scroll a bit.

In [1]:
!python /home/ubuntu/boda2/src/train.py \
  --data_module=MPRA_DataModule \
    --datafile_path=gs://tewhey-public-data/CODA_resources/MPRA_ALL_HD_v2.txt \
    --sep space --sequence_column nt_sequence \
    --activity_columns K562_mean HepG2_mean SKNSH_mean \
    --stderr_columns lfcSE_k562 lfcSE_hepg2 lfcSE_sknsh \
    --synth_val_pct=0.0 --synth_test_pct=99.98 \
    --batch_size=1076 --duplication_cutoff=0.5 --std_multiple_cut=6.0 \
    --val_chrs 7 13 --test_chrs 9 21 X \
    --padded_seq_len=600 --use_reverse_complements=True --num_workers=8 \
  --model_module=BassetBranched \
    --input_len 600 \
    --conv1_channels=300 --conv1_kernel_size=19 \
    --conv2_channels=200 --conv2_kernel_size=11 \
    --conv3_channels=200 --conv3_kernel_size=7 \
    --linear_activation=ReLU --linear_channels=1000 \
    --linear_dropout_p=0.11625456877954289 \
    --branched_activation=ReLU --branched_channels=140 \
    --branched_dropout_p=0.5757068086404574 \
    --n_outputs=3 --n_linear_layers=1 \
    --n_branched_layers=3 --n_branched_layers=3 \
    --use_batch_norm=True --use_weight_norm=False \
    --loss_criterion=L1KLmixed --beta=5.0 \
    --reduction=mean \
  --graph_module=CNNTransferLearning \
    --parent_weights=gs://tewhey-public-data/CODA_resources/my-model.epoch_5-step_19885.pkl \
    --frozen_epochs=0 \
    --optimizer=Adam --amsgrad=True \
    --lr=0.0032658700881052086 --eps=1e-08 --weight_decay=0.0003438210249762151 \
    --beta1=0.8661062881299633 --beta2=0.879223105336538 \
    --scheduler=CosineAnnealingWarmRestarts --scheduler_interval=step \
    --T_0=4096 --T_mult=1 --eta_min=0.0 --last_epoch=-1 \
    --checkpoint_monitor=entropy_spearman --stopping_mode=max \
    --stopping_patience=30 --accelerator=gpu --devices=1 --min_epochs=1 --max_epochs=3 \
    --precision=16 --default_root_dir=/tmp/output/artifacts \
    --artifact_path=/tmp/

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
--------------------------------------------------

K562 | top cut value: 10.54, bottom cut value: -6.43
HepG2 | top cut value: 9.78, bottom cut value: -5.74
SKNSH | top cut value: 10.36, bottom cut value: -6.4

Number of examples discarded from top: 0
Number of examples discarded from bottom: 0

Number of examples available: 666435

--------------------------------------------------

Padding sequences... 

Creating train/val/test datasets with tokenized sequences... 

--------------------------------------------------

Number of examples in train: 1691500 (253.81%)
Number of examples in val:   55841 (8.38%)
Number of examples in test:  45330 (6.8%)

Excluded from train: -1126236 (-168.99)%
--------------------------------------------------
Copying gs://tewhey-public-data/CODA_resources/m

In [2]:
files = !ls /tmp/

_ = [ print(p) for p in files if 'model_artifacts' in p ]
model_artifact = [ p for p in files if 'model_artifacts' in p ][-1]

model_artifacts__20240216_170930__404251.tar.gz


## Design sequences

### Fast SeqProp
We collected the path of the model artifact generated in the last step and will pass it to bash using `--model_artifact /tmp/{model_artifact}`. Probably change this for your deployment.

In [3]:
!python /home/ubuntu/boda2/src/generate.py \
    --params_module StraightThroughParameters \
        --batch_size 256 --n_channels 4 \
        --length 200 --n_samples 10 \
        --use_norm True --use_affine False \
    --energy_module MinGapEnergy \
        --target_feature 0 --bending_factor 1.0 --a_min -2.0 --a_max 6.0 \
        --model_artifact /tmp/{model_artifact} \
    --generator_module FastSeqProp \
         --n_steps 200 --learning_rate 0.5 \
    --energy_threshold -0.5 --max_attempts 40 \
    --n_proposals 1000 \
    --proposal_path /tmp/test__k562__fsp


archive unpacked in /tmp/tmpeu9hu1zu
Loaded model from 20240216_170930 in eval mode
Starting round: 0, generate 1000 proposals
Steps: 100%|██████████████| 200/200 [00:52<00:00,  3.83it/s, Loss=-5.3, LR=1e-6]
finished round
Proposals deposited at:
	/tmp/test__k562__fsp__20240216_171503__555961.pt


In [4]:
import torch
fsp_props = !ls /tmp/
fsp_props = [ p for p in fsp_props if 'test__k562__fsp' in p ][-1]
torch.load(f'/tmp/{fsp_props}')

{'proposals': [{'states': tensor([[[-3.7070, -3.2406, -3.8673,  ..., -0.4899, -2.4676,  4.9513],
            [ 9.2152, 10.3142, -2.7897,  ..., -0.5026,  4.1187, -1.9516],
            [-1.5741, -2.1167, -3.4816,  ...,  0.0254, -0.0922, -1.7666],
            [-0.9142, -2.0754,  7.5319,  ...,  0.7825,  0.1338, -3.1252]],
   
           [[-4.5731, -2.7901, -3.8281,  ..., -2.0469, -2.8894,  9.8387],
            [ 3.1643, 11.9951, -4.1636,  ..., -1.4933, -3.2741, -3.3825],
            [ 7.4364, -3.3401, -4.8432,  ..., -5.3925, 10.0816, -4.7574],
            [-1.1955, -2.1979,  9.3587,  ..., 10.6572, -5.7531, -2.0952]],
   
           [[-4.0923, -3.3490, -3.7582,  ..., -0.1841, -2.2370,  3.1349],
            [10.1623, 12.5539, -2.6694,  ..., -4.5843, 10.3012, -0.1835],
            [-2.1776, -2.9983, -3.7627,  ...,  4.0314, -2.8292,  0.5169],
            [ 0.4006, -1.6453,  9.8911,  ..., -0.3800, -1.7853, -3.3947]],
   
           ...,
   
           [[-3.6596, -3.1684, -3.7423,  ...,  0.0442,

### Simulated Annealing

In [5]:
!python /home/ubuntu/boda2/src/generate.py \
    --params_module BasicParameters \
        --batch_size 256 --n_channels 4 \
        --length 200 \
    --energy_module MinGapEnergy \
        --target_feature 0 --bending_factor 0.0 --a_min -2.0 --a_max 6.0 \
        --model_artifact /tmp/{model_artifact} \
    --generator_module SimulatedAnnealing \
         --n_steps 2000 --n_positions 5 \
         --a 1.0 --b 1.0 --gamma 0.501 \
    --energy_threshold -0.5 --max_attempts 40 \
    --n_proposals 1000 \
    --proposal_path /tmp/test__k562__sa


archive unpacked in /tmp/tmpuya_wra9
Loaded model from 20240216_170930 in eval mode
Starting round: 0, generate 1000 proposals
collect samples
100%|███████████████████████████████████████| 2000/2000 [00:34<00:00, 58.25it/s]
attempt 1 acceptance rate: 256/256
collect samples
100%|███████████████████████████████████████| 2000/2000 [00:31<00:00, 63.05it/s]
attempt 2 acceptance rate: 256/256
collect samples
100%|███████████████████████████████████████| 2000/2000 [00:31<00:00, 63.12it/s]
attempt 3 acceptance rate: 256/256
collect samples
100%|███████████████████████████████████████| 2000/2000 [00:31<00:00, 63.01it/s]
attempt 4 acceptance rate: 256/256
finished round
Proposals deposited at:
	/tmp/test__k562__sa__20240216_171729__147400.pt


In [6]:
import torch
sa_props = !ls /tmp/
sa_props = [ p for p in sa_props if 'test__k562__sa' in p ][-1]
torch.load(f'/tmp/{sa_props}')

{'proposals': [{'proposals': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 1., 0.],
            [1., 0., 0.,  ..., 0., 0., 1.],
            [0., 1., 1.,  ..., 1., 0., 0.]],
   
           [[1., 0., 0.,  ..., 0., 1., 0.],
            [0., 1., 0.,  ..., 0., 0., 1.],
            [0., 0., 1.,  ..., 1., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
   
           [[1., 0., 0.,  ..., 1., 0., 0.],
            [0., 1., 0.,  ..., 0., 0., 1.],
            [0., 0., 1.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 1., 0.]],
   
           ...,
   
           [[1., 0., 1.,  ..., 0., 0., 1.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 1., 0.,  ..., 1., 1., 0.]],
   
           [[0., 0., 0.,  ..., 0., 1., 0.],
            [0., 0., 1.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [1., 1., 0.,  ..., 1., 0., 1.]],
   
           [[0., 0., 0.,  ..., 0., 1., 0.],
     